In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

class FFClassifier(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 1000x1000x3 image tensor)
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        # convolutional layer (sees 16x500x500 tensor)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
        # convolutional layer (sees 32x250x250 tensor)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        # convolutional layer (sees 64x125x125 tensor)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        # convolutional layer (sees 64x64x64 tensor)        
        self.conv5 = nn.Conv2d(32, 64, 3, padding=1)
        # convolutional layer (sees 128x32x32 tensor)

        self.conv6 = nn.Conv2d(64, 64, 3, padding=1)

        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        # linear layer (128 * 16 * 16 -> 500)
        self.fc1 = nn.Linear(64 * 15 * 15, 1024)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(1024, 102)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)        
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv4(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv5(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv6(x)))
        # flatten image input
        x = x.view(-1, 64 * 15 * 15)
        # add dropout layer
#         x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x



In [3]:
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)

    model = models.vgg16(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = FFClassifier(14400, checkpoint['hidden'], 102)

    model.load_state_dict(checkpoint['state_dict'])

    return model



In [4]:
# Load your model to this variable
model = load_checkpoint('./models/model_flower_v2.pt')
   
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 1000
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.5, 0.5, 0.5]
norm_std = [0.5, 0.5, 0.5]

KeyError: 'hidden'